In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
from string import printable

In [ ]:
def remove_emoji(df):
  return df.applymap(lambda y: ''.join(filter(lambda x: x in printable, y)))

In [ ]:
def json_to_df(path,other_person):
  
  sample_message = open(path)
  
  sample_message = json.load(sample_message)
  sample_message = sample_message["messages"]

  new_sample = []
  for m in sample_message:
    if "content" in m.keys():
      new_sample.append(m)
  
  pairs = []
  message = []
  for m in new_sample[::-1]:
    if len(message) == 0 or len(message) == 2:
      if m["sender_name"] == other_person:
        message = []
        message.append(m["content"])
    elif len(message) == 1:
      if m["sender_name"] == other_person:
        message[0] = message[0] + " " + m["content"]
      else:
        message.append(m["content"])
    else:
      if m["sender_name"] == "Ashutosh Singh":
        message[1] = message[1] + " " + m["content"]

    pairs.append(message)

  new_pair = []
  for pair in pairs:
    if len(pair) == 2:
      new_pair.append(pair)  

  me = []
  other = []

  for pair in new_pair:
    other.append(pair[0])
    me.append(pair[1])

  sample_df = pd.DataFrame()
  sample_df["other"] = np.array(other)
  sample_df["me"] = np.array(me)
  sample_df = sample_df.drop_duplicates(keep="first")

  df = remove_emoji(sample_df)
  return df                  

In [ ]:
!unzip ayush.zip

Archive:  ayush.zip
   creating: ayush/
  inflating: ayush/message_1.json    


In [ ]:
username1_df = json_to_df("Name/message_1.json","Username")
username2_df = json_to_df("Name/message_1.json","Username")

In [ ]:
username3_df = json_to_df("Name/message_1.json","Username3")
username3_df = json_to_df("Name/message_1.json","Username4")
username4_df = json_to_df("Name/message_1.json","Username5")
username5_df = json_to_df("Name/message_1.json","Username6")
username6_df = json_to_df("Name/message_1.json","Username7")

In [ ]:
username7_df = json_to_df("Name/message_1.json","Username8")

In [ ]:
train_df = pd.concat([username1_df,username2_df,username3_df,username4_df,username5_df,username6_df,username7_df],axis=0)

In [ ]:
train_df.drop(train_df[train_df["other"]==" "].index,inplace=True)

In [ ]:
train_df.drop(train_df[train_df["me"]==" "].index,inplace=True)

In [ ]:
train_df.shape

(6000, 2)

In [ ]:
import unicodedata
import re
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
train_df["other"] = train_df["other"].apply(lambda x: normalizeString(x))

In [ ]:
train_df["me"] = train_df["me"].apply(lambda x: normalizeString(x))

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
MAX_LENGTH = 10
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

In [ ]:
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(pairs):
    input_lang, output_lang = Lang("other"),Lang("me")
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
pairs = train_df.values

In [ ]:
input_lang, output_lang, pairs = prepareData(pairs)

Read 6000 sentence pairs
Trimmed to 3472 sentence pairs
Counting words...
Counted words:
other 3657
me 2005


In [ ]:
import random
print(random.choice(pairs))

['kk gussa aarha h' 'chodh n be']


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

0m 53s (- 12m 25s) (5000 6%) 3.7082
1m 42s (- 11m 6s) (10000 13%) 3.4322
2m 32s (- 10m 11s) (15000 20%) 3.1157
3m 23s (- 9m 20s) (20000 26%) 2.8152
4m 16s (- 8m 33s) (25000 33%) 2.4791
5m 11s (- 7m 46s) (30000 40%) 2.0934
6m 6s (- 6m 58s) (35000 46%) 1.7638
7m 1s (- 6m 8s) (40000 53%) 1.3684
7m 58s (- 5m 19s) (45000 60%) 1.0777
8m 55s (- 4m 27s) (50000 66%) 0.8172
9m 54s (- 3m 36s) (55000 73%) 0.6790
10m 52s (- 2m 43s) (60000 80%) 0.5137
11m 50s (- 1m 49s) (65000 86%) 0.4166
12m 48s (- 0m 54s) (70000 93%) 0.3221
13m 47s (- 0m 0s) (75000 100%) 0.2794


In [ ]:
device

device(type='cuda')

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> aaj hi frnd banaaunga usko jaldii bol meko bheje
= dher mt pagala
< dher mt pagala <EOS>

> jan gaye h rozz rozz ka h
= ha
< ha <EOS>

> abe kyaa reh gyaa aaa
= bs nikl gya hu
< bs nikl gya hu <EOS>

> ha ankit bola h ayega
= acha
< acha <EOS>

> i am back
= toh ka kru
< toh ka kru <EOS>

> matalabi
= me bhi krta hu khaane me mdat
< me bhi krta hu khaane me mdat <EOS>

> sonali accept krrlii mera req .
= mera nhi ki be
< mera nhi ki be <EOS>

> kyu maja nahi aata hai tumko dosto ke saath
= nhi mere ko ab nhi aata
< nhi mere ko ab nhi aata <EOS>

> ha . .maan li . .ji
= paper kaisa ja rha h
< paper kaisa ja rha h <EOS>

> ha . .ji . .
= mst ladka h
< mst ladka h <EOS>



In [ ]:
def evaluateinput(encoder, decoder,input):
  output_words, attentions = evaluate(encoder, decoder, input)
  output_sentence = ' '.join(output_words)
  print("Input = ", input)
  print('Output = ', output_sentence)
  print('')

In [ ]:
evaluateinput(encoder1,attn_decoder1,"hii")

Input =  hii
Output =  aur btao kya haal h <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"sona mt be")

Input =  sona mt be
Output =  acha thik h <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"koi gf h")

Input =  koi gf h
Output =  ha <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"kya kr rha")

Input =  kya kr rha
Output =  padh rhe the <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"padh liye")

Input =  padh liye
Output =  nhi aaj mja lenge <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"chutiya ho kya")

Input =  chutiya ho kya
Output =  virat kholi ka h n aaaa thuuu <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"kya ho rha")

Input =  kya ho rha
Output =  kuch nhi yr <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"kya haal h")

Input =  kya haal h
Output =  ka btaye <EOS>



In [ ]:
evaluateinput(encoder1,attn_decoder1,"bc")

Input =  bc
Output =  kya bol rhe ho bkchod <EOS>

